In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
import os
import time
import numpy as np
import torch
import torch.nn.functional as F
import pytorch_lightning as pl
import torchmetrics

from torch.utils.data import TensorDataset, DataLoader
from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

import warnings
warnings.filterwarnings("ignore")

# Project imports
import utils
import model_DGDCN

In [ ]:
def accuracy(pred, y):
    """
    :param pred: predictions
    :param y: ground truth
    :return: accuracy, defined as 1 - (norm(y - pred) / norm(y))
    """
    return 1 - torch.linalg.norm(y - pred) / torch.linalg.norm(y)

In [ ]:
# Config
def get_config():
    args = {
        'no_cuda': False,
        'epochs': 1000,
        'lr': 0.001537, 
        'weight_decay': 0.0,
        'dropout': 0.5,
        'conv_hidden': 64,
        'gru_hidden': 451,
        'train_len': 6,
        'pred_len': 6,
        'batch_size': 24,
        'GDC_sum_len': 4,
        'GDC_cut': 0.5,
        'GDC_alpha': 0.05,
        'dijkstra_coeff': 3.5,
        'coeff_identity' : 0.8
        }
    return args

In [ ]:
# Data Loading
feat = utils.load_dataset('../input_data/speed_data.csv')
in_adj = utils.load_adjacency('../input_data/inflow_adj_data.npy')
out_adj = utils.load_adjacency('../input_data/outflow_adj_data.npy')
dijkstra = utils.load_dijkstra('../input_data/dijkstra_matrix.csv')

glob = {
    'max_val_speed' : np.max(feat),
    'min_val_speed' : np.min(feat)
}

In [ ]:
# Train / Eval Pipeline
def run_experiment():
    args = get_config()
    
    # Device Setting
    args['cuda'] = not args['no_cuda'] and torch.cuda.is_available()
    device = torch.device('cuda' if args['cuda'] else 'cpu')
        
    # Dijkstra Matrices
    dijkstra_matrix = utils.generate_dijkstra(dijkstra, coeff = args['dijkstra_coeff'])
    dijkstra_matrix = torch.as_tensor(dijkstra_matrix, dtype = torch.float32, device = device)
        
    # Dataset Splits
    train_X, train_Y, val_X, val_Y, test_X, test_Y = utils.generate_dataset(feat,
                                                                            train_len=args['train_len'],
                                                                            pred_len=args['pred_len'],
                                                                            total_len=None,
                                                                            split_ratio=(0.7, 0.8),
                                                                            normalize=True)
    train_in_adj, val_in_adj, test_in_adj = utils.generate_adjacency(in_adj,
                                                                     train_len=args['train_len'],
                                                                     pred_len=args['pred_len'],
                                                                     total_len=None,
                                                                     split_ratio=(0.7, 0.8),
                                                                     coeff_identity=args['coeff_identity'])
    train_out_adj, val_out_adj, test_out_adj = utils.generate_adjacency(out_adj,
                                                                        train_len=args['train_len'],
                                                                        pred_len=args['pred_len'],
                                                                        total_len=None,
                                                                        split_ratio=(0.7, 0.8))

    train_in_S = utils.GDC(train_in_adj, sum_len = args['GDC_sum_len'], alpha = args['GDC_alpha'], cut_range = args['GDC_cut'])
    val_in_S = utils.GDC(val_in_adj, sum_len = args['GDC_sum_len'], alpha = args['GDC_alpha'], cut_range = args['GDC_cut'])
    test_in_S = utils.GDC(test_in_adj, sum_len = args['GDC_sum_len'], alpha = args['GDC_alpha'], cut_range = args['GDC_cut'])
    train_out_S = utils.GDC(train_out_adj, sum_len = args['GDC_sum_len'], alpha = args['GDC_alpha'], cut_range = args['GDC_cut'])
    val_out_S = utils.GDC(val_out_adj, sum_len = args['GDC_sum_len'], alpha = args['GDC_alpha'], cut_range = args['GDC_cut'])
    test_out_S = utils.GDC(test_out_adj, sum_len = args['GDC_sum_len'], alpha = args['GDC_alpha'], cut_range = args['GDC_cut'])
    
    # Model
    num_nodes = train_in_S.shape[1]
    model = model_DGDCN.DGDCN(metricnormalization = glob['max_val_speed'] - glob['min_val_speed'],
                              dijkstra = dijkstra_matrix,
                              conv_channels = args['conv_hidden'],
                              num_nodes = num_nodes,
                              pred_len = args['pred_len'],
                              out_channels = num_nodes,
                              dropout = args['dropout'],
                              lr = args['lr'],
                              weight_decay = args['weight_decay']).to(device)

    # DataLoaders Setting
    def to_tensor(x):
        return torch.as_tensor(x, dtype = torch.float32, device = device)
    
    train_dataset = TensorDataset(to_tensor(train_X),
                                  to_tensor(train_Y),
                                  to_tensor(train_in_S),
                                  to_tensor(train_out_S))
    val_dataset = TensorDataset(to_tensor(val_X),
                                to_tensor(val_Y),
                                to_tensor(val_in_S),
                                to_tensor(val_out_S))
    test_dataset = TensorDataset(to_tensor(test_X),
                                 to_tensor(test_Y),
                                 to_tensor(test_in_S),
                                 to_tensor(test_out_S))

    train_loader = DataLoader(train_dataset, batch_size = args['batch_size'])
    val_loader = DataLoader(val_dataset, batch_size = args['batch_size'])
    test_loader = DataLoader(test_dataset, batch_size = len(test_dataset))
    
    # Callbacks / Trainer
    checkpoint_callback = ModelCheckpoint(filename = 'epoch{epoch:02d}',
                                          verbose = True,
                                          save_last = True,
                                          monitor = 'val_RMSE',
                                          mode = 'min')
    early_stop_callback = EarlyStopping(monitor = 'val_RMSE',
                                        min_delta = 1e-4,
                                        patience = 100,
                                        verbose = True,
                                        mode = 'min')
    
    AVAIL_GPUS = min(1, torch.cuda.device_count())
    trainer = Trainer(accelerator = 'gpu' if args['cuda'] else 'cpu',
                      devices = AVAIL_GPUS if args['cuda'] else None,
                      max_epochs=args['epochs'],
                      callbacks = [checkpoint_callback, early_stop_callback],
                      deterministic = True)
    
    # Train 
    t1 = time.time()
    trainer.fit(model, train_loader, val_loader)
    print(f'Training Time : {time.time() - t1:.2f} s')
    
    # Predict
    torch.cuda.empty_cache()
    t2 = time.time()
    preds = trainer.predict(model, dataloaders=test_loader,
                            return_predictions=True, ckpt_path = 'best')
    print(f"Prediction Time : {time.time() - t2:.2f} s")
    
    return preds

In [ ]:
# Run
preds = run_experiment()

In [ ]:
# Evaluation
_, _, _, _, _, ground_truth = utils.generate_dataset(feat, train_len=6, pred_len=6, total_len=None,
                                                     split_ratio=(0.7, 0.8), normalize=False)
ground_truth = torch.as_tensor(ground_truth, dtype = torch.float32)
preds0 = (preds[0] * (glob['max_val_speed'] - glob['min_val_speed'])) + glob['min_val_speed']

def evaluation_prediction(predictions, ground_truth, times: list = [1, 2, 3, 4, 5, 6]):
    eval_dict = {}
    mae = torchmetrics.MeanAbsoluteError()
    mape = torchmetrics.MeanAbsolutePercentageError()
    
    for i in times:
        local_dict = {}        
        preds = predictions[:, i-1, :]
        target = ground_truth[:, i-1, :]
        
        local_dict['RMSE'] = torch.sqrt(F.mse_loss(preds, target))
        local_dict['MAE'] = mae(preds, target)
        local_dict['MAPE'] = mape(preds, target)
        local_dict['Accuracy'] = accuracy(preds, target)
        
        eval_dict[i] = local_dict
    
    total = {}
    total['RMSE'] = torch.sqrt(F.mse_loss(predictions, ground_truth))
    total['MAE'] = mae(predictions, ground_truth)
    total['MAPE'] = mape(predictions, ground_truth) 
    total['Accuracy'] = accuracy(predictions, ground_truth)
    eval_dict['total'] = local_dict
        
    return eval_dict

In [ ]:
eval_dict = evaluation_prediction(preds0, ground_truth)
# eval_dict
print(eval_dict[1])
print(eval_dict[3])
print(eval_dict[6])